In [1]:
pip install gspread oauth2client pandas google-auth google-auth-oauthlib google-auth-httplib2


Note: you may need to restart the kernel to use updated packages.


In [3]:
import datetime
import requests
import os
import json
import pandas as pd
import pygsheets
import configparser

from authlib.integrations.requests_client import OAuth2Session
from google.oauth2.service_account import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle

# Loads configuration from confi.txt
config = configparser.ConfigParser()
config.read("/Users/yashasvi.reddy/Documents/adobe-aep-api-main/confi.txt")  # Ensure the path is correct

# Checks if the 'oauths2s' section exists
if 'oauths2s' not in config:
    raise KeyError("Missing 'oauths2s' section in confi.txt file")

# Checks if the required keys exist in the 'oauths2s' section
if 'client_id' not in config['oauths2s'] or 'client_secret' not in config['oauths2s']:
    raise KeyError("Missing 'client_id' or 'client_secret' in 'oauths2s' section of confi.txt file")

client_id = config["oauths2s"]["client_id"]
client_secret = config["oauths2s"]["client_secret"]

# Authentication and API request setup
aep_scopes = (
    "openid, AdobeID, read_organizations, additional_info.projectedProductContext, session"
)
token_endpoint = "https://ims-na1.adobelogin.com/ims/token/v3"
org_id = "10D31225525FF5790A490D4D@AdobeOrg"

# Sandbox ID
sandbox = "pplus-prod"
output_dir = "output/"
# Ensures the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Step 2: Getting the OAuth2 token
client = OAuth2Session(client_id=client_id, client_secret=client_secret, scope=aep_scopes)
token = client.fetch_token(token_endpoint, grant_type="client_credentials")
access_token = token.get("access_token")

# Step 3: Fetching segment definitions
url = "https://platform.adobe.io/data/core/ups/segment/definitions"
filename = "segment_definitions"

params = {
    "limit": 100,
    "orderBy": "desc:created",
}

# Making the request to the Segmentation Service API
r = requests.get(
    url=url,
    params=params,
    headers={
        "Authorization": f"Bearer {access_token}",
        "x-api-key": client_id,
        "x-gw-ims-org-id": org_id,
        "x-sandbox-name": sandbox,
        "accept-encoding": "gzip",
        "Accept": "application/json",
    },
)

# Step 4: Processing and saving the response
if r.status_code == 200:
    res = r.json()
    with open(output_dir + filename + ".json", "w") as f:
        json.dump(res, f, indent=4)
    if 'segments' in res:
        df = pd.DataFrame(res['segments'])
    else:
        df = pd.DataFrame(res)
else:
    print("Error code", r.status_code)
    print("Error Description", r.json())

# Specify the columns to keep
columns_to_keep = [
    "id", "ttlInDays", "sandbox", "name", "description", "evaluationInfo", "creationTime",
    "updateEpoch", "updateTime", "createEpoch", "_etag", "dependents", "definedOn",
    "dependencies", "createdBy", "lastModifiedBy", "lifecycle", "namespace", "audienceId",
    "saveSegmentMembership", "origin", "metrics", "tags", "lifecycleState", "originName"
]

# Filter the DataFrame to keep only the specified columns
df_filtered = df[columns_to_keep]

# Displaying the first 10 rows of the filtered DataFrame
print(df_filtered.head(10))

# Path to the client_secret.json file
client_secret_path = '/Users/yashasvi.reddy/Documents/adobe-aep-api-main/client_secret_58196927069-pt0646otevl7unvkhoc0n5qce21aup3m.apps.googleusercontent.com.json'

# Authenticate with Google Sheets using OAuth2 in an interactive environment
SCOPES = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']

creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(client_secret_path, SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

# Authenticate with Google Sheets
gc = pygsheets.authorize(custom_credentials=creds)

# Creates or opens the Google Sheet
spreadsheet_name = "Segment Definitions"
try:
    sh = gc.create(spreadsheet_name)
    sh.share('stef.work@paramount.com', role='writer') # Share with your email for access
except Exception as e:
    sh = gc.open(spreadsheet_name)

# Selects the first sheet
worksheet = sh.sheet1

# Clears the existing content in the sheet
worksheet.clear()

# Writes the filtered DataFrame to the Google Sheet
worksheet.set_dataframe(df_filtered, (1, 1))

print(f"Data has been exported to Google Sheet: {spreadsheet_name}")


                                     id  ttlInDays  \
0  140b12d4-e2ad-4ccf-b506-adc30d132093         30   
1  a6e510dd-6710-4955-8bab-fe1b724e07ec         30   
2  0409f968-d8fb-452f-b43e-ff0b2efde521         30   
3  0db41ab7-6dcf-4f7d-9bfe-6bf9e9ac19d1         30   
4  198414d3-c011-4de6-8f85-31432d38e787         30   
5  ad525b61-20e8-480f-9fa3-7c8eb87a112a         30   
6  3dc8b31c-477d-4e21-8b90-a7d17a27da47         30   
7  437d8078-1e51-46aa-9d37-a32a3a3cd872         30   
8  f057334e-16d8-466c-b46e-47d083879a37         30   
9  d40a22ed-4290-417f-9a5a-5661b2ebd44f         30   

                                             sandbox  \
0  {'sandboxId': '56976b61-e0b3-4b2f-976b-61e0b3d...   
1  {'sandboxId': '56976b61-e0b3-4b2f-976b-61e0b3d...   
2  {'sandboxId': '56976b61-e0b3-4b2f-976b-61e0b3d...   
3  {'sandboxId': '56976b61-e0b3-4b2f-976b-61e0b3d...   
4  {'sandboxId': '56976b61-e0b3-4b2f-976b-61e0b3d...   
5  {'sandboxId': '56976b61-e0b3-4b2f-976b-61e0b3d...   
6  {'sandboxI

/opt/anaconda3/lib/python3.12/site-packages/pygsheets/worksheet.py:1430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('unicode').replace('<NA>', nan)
/opt/anaconda3/lib/python3.12/site-packages/pygsheets/worksheet.py:1430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('unicode').replace('<NA>', nan)
/opt/anaconda3/lib/python3.12/site-packages/pygsheets/worksheet.py:1430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

Data has been exported to Google Sheet: Segment Definitions
